### - Exercici 1

Crea una base de dades relacionals senzilla utilitzant MySQL(https://www.mysql.com/) i connecta-la a Python

In [55]:
import mysql.connector

In [56]:
#connect to mysql server and the database napster
mydb = mysql.connector.connect(host = 'localhost',
                              user = 'root',
                              password = 'Armeleta',
                              database = 'napster',
                              use_pure = True ) #Avoid SSL error

### Nivell 2

- Exercici 2

Mostra que pots carregar algunes consultes senzilles a un Pandas Dataframe.

In [57]:
mc = mydb.cursor(buffered = True)
mc.execute('show tables')

tableNames = []
#Show the names of the tables
for table in mc:
    tableNames.append(table[0])

print(tableNames)

['artist', 'genre', 'song']


In [58]:
def printID_values(colname, mycursor):
    mycursor.execute('select * from ' + colname)
    
    entries = mycursor.fetchall()
    
    for valueID, *values in entries:        
        for count, value in enumerate(values):
            if type(value) is not 'str':
                values[count] = str(value)
        print(valueID, '->', ', '.join(values))
    

In [59]:
#Let's check the artists table
printID_values('artist', mc)

1 -> Queen
2 -> Nirvana
3 -> Spice Girls
4 -> Salt and Pepa
5 -> Pendulum


In [60]:
#check genres table
printID_values('genre', mc)

1 -> Rock
2 -> Pop
3 -> Hip hop
4 -> Drum and bass


In [61]:
#check the songs table
printID_values('song', mc)

1 -> Don't Stop me now, 1, 1
2 -> Bohemian Rhapsody, 1, 1
3 -> Smells like teen spirit, 2, 1
4 -> Bycicle, 1, 2
5 -> Wannabe, 3, 2
6 -> Leero (Concert), 1, 2
7 -> Come as you are, 2, 1
8 -> Friend, 3, 2
9 -> Shoop, 4, 3
10 -> Say you'll be there, 3, 2
11 -> Push it, 4, 3
12 -> Self vs Self, 5, 4
13 -> I want to break free, 1, 1


We do not know what the numbers next to the song names are so:

In [62]:
mc.execute('select * from song')

print('Names of columns in the song table')

for colTuple in mc.description:
    print(colTuple[0])

Names of columns in the song table
idSONG
NAME
idARTIST
idGENRE


Let's make some joins:

In [78]:
def songFinder(search_criteria, search_field, myCursor):
    
    if search_criteria == 'song':
        myCursor.execute(f'select aux.name, artist.name, genre.name\
                           from ((( select * \
                                  from song \
                                  where name like "{search_field}") as aux \
                           left join artist on aux.idartist = artist.idartist)\
                           left join genre on aux.idgenre = genre.idgenre)')
        
        songTuple = myCursor.fetchall()
        
        print('Song:', songTuple[0][0], '// Artist:', songTuple[0][1],
              '// Genre:', songTuple[0][2])
        
    elif search_criteria == 'artist':
        myCursor.execute(f'select song.name, auxtab.name\
                          from (song \
                          right join (select * from artist \
                                      where name = "Queen") as auxtab \
                          on song.idartist = auxtab.idartist)')
        
        artistTuple = myCursor.fetchall()
        for item in artistTuple:
            print(item)
    

In [79]:
#Find the son 'self vs self and its info'
songFinder('song', 'self vs self', mc)

Song: Self vs Self // Artist: Pendulum // Genre: Drum and bass


In [80]:
#find all songs by Queen
songFinder('artist', 'Queen', mc)

("Don't Stop me now", 'Queen')
('Bohemian Rhapsody', 'Queen')
('Bycicle', 'Queen')
('Leero (Concert)', 'Queen')
('I want to break free', 'Queen')


Finally let's make a join of all the songs and put them into a dataframe

In [83]:
#Create a table containing all the names of songs/artist/genre ordered by genre
mc.execute('select song.name, artist.name, genre.name \
            from ((song left join artist on song.idartist = artist.idartist) \
            left join genre on song.idgenre = genre.idgenre) \
            order by genre.name')

allSongs = mc.fetchall()
print(allSongs)

[('Self vs Self', 'Pendulum', 'Drum and bass'), ('Shoop', 'Salt and Pepa', 'Hip hop'), ('Push it', 'Salt and Pepa', 'Hip hop'), ('Bycicle', 'Queen', 'Pop'), ('Wannabe', 'Spice Girls', 'Pop'), ('Leero (Concert)', 'Queen', 'Pop'), ('Friend', 'Spice Girls', 'Pop'), ("Say you'll be there", 'Spice Girls', 'Pop'), ("Don't Stop me now", 'Queen', 'Rock'), ('Bohemian Rhapsody', 'Queen', 'Rock'), ('Smells like teen spirit', 'Nirvana', 'Rock'), ('Come as you are', 'Nirvana', 'Rock'), ('I want to break free', 'Queen', 'Rock')]


In [86]:
import pandas as pd

df = pd.DataFrame(allSongs, columns = ['Song', 'Artist', 'Genre'])
df

,Song,Artist,Genre
0,Self vs Self,Pendulum,Drum and bass
1,Shoop,Salt and Pepa,Hip hop
2,Push it,Salt and Pepa,Hip hop
3,Bycicle,Queen,Pop
4,Wannabe,Spice Girls,Pop
5,Leero (Concert),Queen,Pop
6,Friend,Spice Girls,Pop
7,Say you'll be there,Spice Girls,Pop
8,Don't Stop me now,Queen,Rock
9,Bohemian Rhapsody,Queen,Rock
